In [51]:
import pickle
import math
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import time
import yfinance as yf

from datetime import date
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm_notebook
from xgboost import XGBRegressor

%matplotlib inline

In [55]:
ticker = "LULU"

In [56]:
with open(f'{ticker}Model.pickle', 'rb') as handle:
    model = pickle.load(handle)

In [57]:
model

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1.0, colsample_bynode=1, colsample_bytree=1.0,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.2, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=13,
             missing=nan, monotone_constraints='()', n_estimators=130, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=100,
             reg_alpha=0, reg_lambda=1, ...)